## **Le fichier *nom_prenom_OPT_M1_TP2*.ipynb est à rendre avant samedi 21 décembre à 15h (heure de Seattle)**

# TP2 : Optimisation sous contraintes

## Patie I Le modèle

Dans la suite nous considérons $\Omega=(0,1)$ et regarderons le problème de minimisation suivant :
\begin{equation}
\label{P}
\min_{u\in\mathcal{K}}\mathcal{J}[u],
\tag{$\mathcal{P}$}
\end{equation}
où $$ \mathcal{J}[u]:=\int_\Omega (\frac{1}{2}u'(x)^2-f(x)u(x))dx $$
et
$$ \mathcal K:=\{ u\in H^1(\Omega)\;|\;u(0)=a,u(1)=b,\; u(x)\geq g(x)\}, $$
où $f$ et $g$ sont deux fonctions continues sur $\Omega$.


Dans ce TP on veut se focaliser sur  la méthode du **gradient  projeté** et celle de la **pénalisation extérieure**.

Comme dans le TP 1 on discrétise le problème par élements finis. On considère un maillage uniforme de $\Omega$ donné par $0=x_0<x_1<\cdots<x_{N+1}=1$ avec $h=x_{i+1}-x_{i}$ et 
$$ \mathcal K_h:=\{ v_h\in\mathcal C^0(\Omega)\;|\; v_h(x)=v_h(x_i)+(x-x_i)\frac{v_h(x_{i+1})-v_h(x_{i})}{h_i}\;\forall x\in[x_i,x_{i+1}]\; v(x_i)\geq g(x_i)\;i=0,\cdots,N+1\;\text{et}\; v_h(0)=a,v_h(1)=b\}.$$

On approche \eqref{P} par le problème suivant:
\begin{equation}
\label{Ph}
\min_{u_h\in\mathcal{K}_h}\mathcal{J}[u_h],
\tag{$\mathcal{P}_h$}
\end{equation}
où
$$ \mathcal{J}[u_h]:=\int_\Omega \Big(\frac{1}{2}u_h'(x)^2-f(x)u_h(x)\Big)dx. $$

**QI.1)** Comme dans le TP1 (**REPETITA IUVANT**) expliciter le problème \eqref{Ph} en utilisant la formule de trapèzes pour approcher les intégrales et montrer qu'il peut se mettre sous la forme 

\begin{equation}
\label{Pn}
\begin{cases}
&\min_{u_N\in \mathcal K_N}\mathcal J_N[u_N] \\
&\mathcal K_N=\{v_N\in\mathbb{R}^N\;|\; v_i\geq g(x_i)\;\forall i \}
\end{cases}
\tag{$\mathcal P_N$}
\end{equation}
où $\mathcal J_N$ est à expliciter. 

**QI.2)** Calculer le gradient $\nabla \mathcal J_N$ de $\mathcal J_N$.

**QI.3)** Montrer que \eqref{Pn} admet une unique solution.

**QI.4)** Implémenter $\mathcal J_N$ et $\nabla \mathcal J_N$ à travers les deux fonction Python
`J(U,a,b,fx)` et `DJ(U,a,b,fx)` (attention aux conditions au bord!!).

**QI.5)** Pour $a=0$, $b=0$, $N=100$, $f(x)=1$, $g(x)=\max(1.5-20(x-0.4)^2,0)$ :
- résoudre \eqref{Pn} en utilisant `scipy.optimize.minimize`

In [ ]:
import numpy as np
from scipy.optimize import minimize
...
''' Definir ou importer f, g, J et DJ '''
...
N = 100

a, b, x = 0, 0, np.linspace(0,1, N + 2)

fx, gx = f(x), g(x)

Jf = lambda u : J(u, a, b, x, fx)

DJf = lambda u : DJ(u, a, b, x, fx)

cons = ({'type': 'ineq','fun' : lambda x: x - gx[1:-1], 'jac' : lambda x: np.eye(np.size(x))})

u = np.zeros(n)

res = minimize(Jf, u, method='SLSQP', jac=DJf, constraints=cons, tol=1e-8,

               options={'xtol': 1e-8, 'disp': True,'maxiter':5000})
# res.x contient la solution

- représenter sur le même graphique la solution ainsi que l'obstacle. Tester aussi $f(x)=\pi^2\sin(\pi x)$


## Parite II Le gradient projeté

L'algorithme du gradient projeté à pas fixe pour minimiser une fonctionelle $\mathcal J_N:\mathbb{R}^N\rightarrow\mathbb{R}$ sur un esemble $\mathcal K _N$ pour un point de départ $\bf u_0$ et un pas $t$  est donné par 

\begin{equation}
\label{GrdPro}
\begin{cases}
& \bf d_k=-\nabla \mathcal J_N[\bf u_k],\\
& \bf u_{k+1}= P_{\mathcal K_N}(\bf u_k+t \bf d_k),
\end{cases}
\tag{$\mathcal{GP}$}
\end{equation}

où $P_{\mathcal K_N}$ désigne la projection sur $\mathcal K_N$.
On arrêtera les itérations lorsque $\parallel \bf u_{k+1}-\bf u_k \parallel \leq \epsilon$.

<div class="alert alert-block alert-danger">
<b>Projection sur un convexe </b>
Soit $K\subseteq \mathbb{R}^d$ un convexe fermé et $x\in \mathbb{R}^d$. Alors il y a equivalence entre 
- $p=P_K(x)$
- $\forall y\in K$ $<p-x,p-y> \leq 0$
</div>

**QII.1)**
<b>Etude théorique : convergece de la méthode</b>
Soit $\mathcal J[\bf u]$ de la forme $\mathcal J[\bf u]=\frac{1}{2}<\bf u, A \bf u>-<\bf b, \bf u>$ où $A$ est une matrice symétrique et définie positive. $\mathcal K_N$ est un convexe fermé non vide. On admet que $x\mapsto P_{\mathcal K_N}(x)$ est 1-Lip par rapport à la norme euclidienne. On veut montrer le théorème suivant :

*Si $t\in(0,\dfrac{2}{\lambda_{max}})$ (où $t$ est le pas de l'algo \eqref{GrdPro} et $\lambda_{max}$ est la plus grande valeur propre de la matrice $A$), alors quel que soit $\bf u_{0}$, la suite $\{\bf u_k\}_{k\geq0}$ définie par le gradient projeté converge vers le minimum $\bf u^\star$*

(Indication : montrer d'abord que $\parallel \bf u_{k+1}-\bf u^\star\parallel \leq C(t)\parallel \bf u_{k}-\bf u^\star\parallel$)


**QII.2)** Montrer que la projection sur $\mathcal K_N$ est donnée par
$$(P_{\mathcal K_N}(\bf u))_i=\max(u_i,g_i)\;\forall i=1,\cdots,N $$
et écrire un fonction `projK(u,gx)` qui prend en argument deux vecteurs `u` et `gx` et qui renvoie un vecteur $P_{\mathcal K_N}(\bf u)$.

**QII.3)** Implementer une fonction `grad_proj(J,DJ,gx,u0,t,epsilon,iterMax)` qui prend en argument  la fonctionnelle `J` à minimiser, le gradient `DJ`, la contrainte `gx`, la valeur initiale `u0`, le pas `t`, le nombre maximal d'iterations autorisées `IterMax`, la tolerance `epsilon`. Cette fonction devra retourner:
- `u` dernier terme de la suite $\bf u_k$ si `store=0` ou tous les termes si `store=1`;

- `iter` nombre d'itérations effectuées.

- `err` liste de $\parallel \bf u_{k+1}-\bf u_k \parallel$.

**QII.4)** Tester la focntion `grad_proj` pour $a=0$, $b=0$, $f(x)=1$, $g(x)=\max(1.5-20(x-0.4)^2,0)$, $N=2,10,20,50,100$, $\epsilon=10^{-5}$. Commenter les résultats obtenus en prennant d'abord $t>\dfrac{2}{\lambda_{max}}$ et puis 
$$t=\dfrac{1}{\lambda_{min}+\lambda_{max}}, $$
$\lambda_{min},\lambda_{max}$ sont respectivement la plus petite et la plus grande valeur propre de la matrice $A$ associée à la fonctionnelle $\mathcal J_N$.
Afficher à l'aide de la fonction `frpintf` le nombre d'itérations ainsi que le temps de calcul pour chaque $N$. Tracer sur une même figure les solutions approchées $\bf u_N$, ainsi que le graphe de la fonction $g$.

**QII.5)** Vérifier que la solution (en faisant varier $N$) satisfait
\begin{equation}
\begin{cases}
&u(x)''\leq f(x),\\
&u(x)\leq g(x),\\
&(-u(x)''-f(x))(u(x)-g(x))=0,
\end{cases}
\end{equation}
sur $\Omega$. La première équation traduit une concavité maximale de la solution. La deuxième équation représente l'obstacle $g$. La troisième traduit le fait que l'on a au moins égalité dans une des deux équations précédentes : soit on résout $-u(x)''=f(x)$, soit $u(x)=g(x)$ et $u$ est sur l'obstacle.

**QII.6)** Reprendre la question précédente pour $f(x)=\pi^2\sin(\pi x)$.


## Partie III Méthode de pénalisation extérieure

La méthode pénalisation extérieure consiste à remplacer le problème sous contrainte \eqref{Pn} par une suite de problèmes sans contraite qui converge vers \eqref{Pn}.

Soit $\gamma > 0$ le paremètre de pénalisation et $\mathcal J_N^\gamma$ la fonctionnelle définie comme 
\begin{equation}
\mathcal J_N^\gamma[\bf u]=\mathcal J_N[\bf u]+\dfrac{1}{\gamma}\sum_{i=1}^N(\max(g_i-u_i,0))^2.
\end{equation}

Le minimiseur de $\mathcal J_N^\gamma$ converge vers le minimiseur de $\mathcal J_N$ quand $\gamma\rightarrow 0$.

**QIII.1)** Calculer le gradient $\nabla \mathcal J_N^\gamma$.

**QIII.2)** Implementer une fonction `penalisation(J,DJ,gx,gamma,u0,t,epsilon,iterMax)` pour trouver le minimiseur de $\mathcal J_N^\gamma$ en utilisant l'algorithme du gradient à pas fixe (`t` est le pas de descente).

**QIII.3)** Tester cette fonction avec les valeurs numériques de (8) et (10) en prenant : $N=20$, $t=0.1$ et $\gamma\in\{10^5,10^4,10^3,10^2,10,1,0.1\}$. Afficher à l'aide de la fonction `fprintf` le nombre d'itérations ainsi que le temps de calcul pour chaque $\gamma$. Tracer sur une même figure les solutions $\bf u^\gamma_N$ ainsi que la fonction $g$ et le graphe de la soltuion de $-u(x)''=1$. Commenter les résultats obtenus pour les différentes valeurs de $\gamma$ et déduire la nécessité d'une méthode variant le paramètre de pénalisation.

**QIII.4)** On va maintenant mettre en place un algorithme permettant d'ajuster le paramètre de pénalisation à chaque itération :
- trouver la solution $\bf u_{k+1}$ approchée de $\mathcal J^{\gamma_k}_N$,
- mettre à jour le paramètre $\gamma_{k+1}=\dfrac{\gamma_k}{100}$.

Programmer alors une nouvelle fonction `penalisation_var(J,DJ,gx,u0,t,epsilon,iterMax)`.

**QIII.5)** Tester cette nouvelle fonction avec les valeurs numériques de (8) et (10) en prenant  $N=20$ et $t=0.1$. Commenter les résultats.